In [ ]:
EBS문장 ebs sentence list로 -> esl 한글로 전처리 -> 띄어쓰기 붙이기 -> 토크나이징

STT는..

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [1]:
from jamo import h2j, j2hcj, j2h

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical
from PreProcessing.Opentext import get_EBS

In [3]:
from soynlp.hangle import levenshtein
from soynlp.hangle import jamo_levenshtein
from soynlp.hangle import compose
from soynlp.hangle import decompose
from konlpy.tag import Kkma
import nltk
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import regexp_tokenize
from nltk.help import upenn_tagset
from string import punctuation
from nltk import Text
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum
import pandas as pd
import numpy as np
import re
from PreProcessing.Dictionary import sd, ad, kopro
from PreProcessing.Opentext import get_EBS
from PreProcessing.Opentext import get_EBS_entered
from PreProcessing.Opentext import get_STT1_Google_entered
from PreProcessing.Opentext import get_STT1_Google
from PreProcessing.Opentext import get_STT1_Transcribe_entered
from collections import defaultdict
pd.set_option("display.max_rows", None, "display.max_columns", None)
from PreProcessing.find_common_part import find_common_part
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum

0
0
0
1
2
5
4
8


In [4]:
EBS = get_EBS(1, 26)

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [5]:
GOOGLE = get_STT1_Google(1, 26)

['lec00001_STT1_Google.txt', 'lec00002_STT1_Google.txt', 'lec00003_STT1_Google.txt', 'lec00004_STT1_Google.txt', 'lec00005_STT1_Google.txt', 'lec00006_STT1_Google.txt', 'lec00007_STT1_Google.txt', 'lec00008_STT1_Google.txt', 'lec00009_STT1_Google.txt', 'lec00010_STT1_Google.txt', 'lec00011_STT1_Google.txt', 'lec00012_STT1_Google.txt', 'lec00013_STT1_Google.txt', 'lec00014_STT1_Google.txt', 'lec00015_STT1_Google.txt', 'lec00016_STT1_Google.txt', 'lec00017_STT1_Google.txt', 'lec00018_STT1_Google.txt', 'lec00019_STT1_Google.txt', 'lec00020_STT1_Google.txt', 'lec00021_STT1_Google.txt', 'lec00022_STT1_Google.txt', 'lec00023_STT1_Google.txt', 'lec00024_STT1_Google.txt', 'lec00025_STT1_Google.txt', 'lec00026_STT1_Google.txt']


In [6]:
EBS

"반갑습니다, EBS 친구 여러분들. 여러분들과 함께 중학뉴런수학3(하)를 강의하게 된 손석민입니다.2학년 뉴런을 통해서도 만났었는데요. 여러분들, 이렇게 다시 만나게 되어서 정말 반갑습니다.앞으로 3(하) 과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거고요.오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 3학년 과정이, (하) 과정이 특히 어떻게 수업되는지 확인할 수 있을 겁니다.일단 3학년 과정 같은 경우에는 크게 3개의 대단원으로 이루어져 있는데요. 첫 번째가 삼각비라는 단원이고 두 번째가 원의 성질, 세 번째가 통계. 이렇게 이루어져 있습니다.우리가 1학년(하)도 마찬가지고 2학년(하)도 그래 왔듯, ‘주로 도형에 대한 이야기가 3(하)에서 많이 다루어지는구나.’ 라는 걸 생각하면 될 테고요.오늘은 그 첫 시간이니까 ‘삼각비.’ ‘삼각’ 알아요. ‘비’ 알아요. 근데 이제 합쳐져 있는 ‘삼각비’라는 새로운 용어가 나왔죠.그럼 이 개념을 공부하기에 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다. 오늘 그 첫 시간, 함께 시작해 보도록 할 텐데요. 중학교 3학년 수학! 손샘과 끝냅시다!오늘 공부할 내용, 바로 두 가지 개념인데요. 오늘 두 가지 개념 첫 번째 제목이 굉장히 길어요. ‘닮은 직각삼각형에서 대응변의 길이의 비’라는 것이 있습니다.그다음 두 번째 개념이 바로 뭐냐면, ‘삼각비’라는 건데 삼각비의 뜻이겠죠? 이 첫 번째 개념은, 앞으로 쭉 공부할 내용이 다 직각삼각형에서 주로 다루어질 텐데직각삼각형에서 갖는 기본적인 성질이 있는데 그 성질을 알고 가자는, ‘삼각비를 공부하기에 앞선 징검다리 내용이다.’ 라고 여러분들이 받아들이시면 됩니다.그렇다면 ‘닮은 직각삼각형에서’라고 했으니까 닮은 직각삼각형을 그려 봐야 되겠죠? 닮은 직각삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면우리는 이렇게 그리면 될 거 같아요. 이렇게 한번 그려 보겠습니다, 이렇게. 여기에서부터 수선

In [7]:
GOOGLE

"반갑습니다 이비에스 여러분들 여러분들과 함께 중학뉴런수학 3할을 강의하기 됨 손성민입니다 2학년 이런을 통해서도 만났었는데 여러분들 이렇게 다시 만나게 되어서 정말 반갑습니다 앞으로 3과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거구요 오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 3학년 과정이 하마 종이접기 어떻게 수업 되는지 확인할 수 있을 겁니다 일단 3학년 가정 같은 경우에는 크게 세 개의대 단원으로 이루어져있는데요 첫 번째가 삼각비라는 단어 이거 두 번째가 원의 성질 세 번째가 통계 이렇게 이루어져 있습니다 우리가 1학년 하도 마찬가지고 2학년 하도 그래왔듯 주로 도형에 대한 이야기가 3화\n많이 다루어 지는구나라는 걸 생각하면 될테구요 오늘은 그 첫 시간이니까 삼각비 삼각 알아요 알아요 근데 이제 합쳐져 있는 삼각비라는 새로운 영화이 개념은 공부하기 앞서서 당연히 삼각비의 뜻부터 시작하는 것이 첫 시간입니다 오늘도 차 시간 함께 시작해 보도록 할 텐데요 중학교 3학년 수학 거 끝냈어\n 오늘 공부한 내용은 바로 두 가지 개념인데요 자 오늘 두 가지 그냥 첫 번째 제목이 굉장히 길어요 닮은 집값 삼각형에서 대응 변의 길이의 피 나는 것이 있습니다 그냥 두 번째 개념이 바로 뭐냐면 나는 건데 생각 뜻이겠죠 2000번째 개념은 두 번째 앞으로 쭉 공부할 내용이다 직각삼각형에서 주로 다루어 질 텐데 직각삼각형에서 가는 기본적인 성질이 있는데 왜 성질을 알고가자 나는 어떤 상각비를 공부하기 앞선 징검다리 내용이라고 여러분들이 받아들이셨습니다 자 그렇다면 닮은 직각삼각형에서 있으니까 닮은 직각삼각형 그려 봐야 되겠죠 닮은 직각 삼각형 두 개를 편하게 그릴 수 있는 제일 좋은 방법은 뭘까 생각해 보면 우리는 이렇게 그러면 될 거 같아요 차 이렇게 한번 그려보겠습니다 이렇게\n 이렇게 여기서부터 수선을 이렇게 커 보겠습니다 일단 바깥쪽에 직각삼각형 않아 보입니다 그죠 또 다른 직각삼각형 하나를 빗변이 1점을 잡아서

In [14]:
def decimalnumber_to_korean(m):
    answer = ''
    for _ in m.group(1):
        answer += kopro[int(_)]
    answer += '점'
    for _ in m.group(3):
        answer += kopro[int(_)]
    return answer

In [15]:
def number_to_korean(s):
    #소수점 한국어로
    s = re.sub(r'(\d+)(\.)(\d+)', decimalnumber_to_korean, s)
    
    #정수 한국어로
    fl = re.findall(r'\d+', s)
    fl = set(fl)
    fl = [int(_) for _ in fl]
    fl.sort(reverse=True)
    fl = [str(_) for _ in fl]
    for n in fl:
        s = s.replace(n, kopro[int(n)])
    return s

In [16]:
def mathsymbol_to_korean(s):
    for ms in sd.keys():
        if ms in s:
            s = s.replace(ms, sd[ms])
    return s

In [17]:
def alphabet_to_korean(s):
    for a in ad.keys():
        if a in s:
            s = s.replace(a, ad[a])
    return s

In [18]:
def remove_newlines(s):
    
    return re.sub(r'[\n]+', ' ', s)

In [53]:
def remove_puncuations(s):
    
    return re.sub(r'[\.?,‘’]+', ' ', s)

In [64]:
def insert_dot(text, dot):
    pattern1 = re.compile(r'(ㅂㄴㅣㄷㅏ)$')   #ㅂ 니다
    pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
    pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
    pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    text_list = []
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    for _ in text.split(' '):         
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
        elif pattern2.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _2 += 1
            #list2.append(_)
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
        elif pattern4.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _4 += 1
        elif pattern5.findall(new_):
            text_list.append(_.replace(_,_+dot))
            _5 += 1
        else:
            text_list.append(_)
            _6 += 1
            #list6.append(_)
    print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list    

In [74]:
# EBS 전처리
def EBS_preprocessing(EBS):
    
#     수학기호 한국어로 변환(EBS)
    ebs_math_conversion = mathsymbol_to_korean(EBS)
        
#     숫자 한국어로 변환(EBS)
    ebs_number_conversion = number_to_korean(ebs_math_conversion)
        
#     알파벳 한국어로 변환(EBS)
    ebs_alphabet_conversion = alphabet_to_korean(ebs_number_conversion)
    
#     특수문자 제거
    ebs_punc_removed = remove_puncuations(ebs_alphabet_conversion)
    
#     점 집어넣기
    ebs_dot_inserted = ' '.join(insert_dot(ebs_punc_removed,'.'))

#     띄어쓰기 제거
    ebs_whitespace_removed = re.sub('[\s]+', '', ebs_dot_inserted)

    return ebs_whitespace_removed

In [75]:
# GOOGLE 전처리
def GOOGLE_preprocessing(GOOGLE):
        
#     개행 삭제
    google_newline_removed = remove_newlines(GOOGLE)
    
#     수학기호 한국어로 변환
    google_math_conversion = mathsymbol_to_korean(google_newline_removed)
    
#     숫자 한국어로 변환(GOOGLE)
    google_number_conversion = number_to_korean(google_math_conversion)
        
#     알파벳 한국어로 변환(GOOGLE)
    google_alphabet_conversion = alphabet_to_korean(google_number_conversion)
    
#     특수문자 제거
    google_punc_removed = remove_puncuations(google_alphabet_conversion)
    
#     점 집어넣기
    google_dot_inserted = ' '.join(insert_dot(google_punc_removed,'.'))
    
#     띄어쓰기 제거
    google_whitespace_removed = re.sub('[\s]+', '', google_dot_inserted)

    return google_whitespace_removed

In [76]:
ko_pre_EBS = EBS_preprocessing(EBS)

pattern1 = 1872, pattern2= 2455, pattern3 = 0, pattern5 = 781, pattern6 = 1128


In [77]:
ko_pre_GOOGLE = GOOGLE_preprocessing(GOOGLE)

pattern1 = 1729, pattern2= 2167, pattern3 = 0, pattern5 = 563, pattern6 = 1109


In [78]:
ko_pre_EBS

'반갑습니다.이비에스친구여러분들여러분들과함께중학뉴런수학삼하를강의하게된손석민입니다.이학년뉴런을통해서도만났었는데요.여러분들이렇게다시만나게되어서정말반갑습니다.앞으로삼하과정어떻게진행되는지에대해서는여러분들께서오리엔테이션강의를통해서확인할수있을거고요.오리엔테이션강의는반드시듣고오셔야여러분들이삼학년과정이하과정이특히어떻게수업되는지확인할수있을겁니다.일단삼학년과정같은경우에는크게삼개의대단원으로이루어져있는데요.첫번째가삼각비라는단원이고두번째가원의성질세번째가통계이렇게이루어져있습니다.우리가일학년하도마찬가지고이학년하도그래왔듯주로도형에대한이야기가삼하에서많이다루어지는구나라는걸생각하면될테고요.오늘은그첫시간이니까.삼각비삼각알아요.비알아요.근데.이제합쳐져있는삼각비라는새로운용어가나왔죠그럼이개념을공부하기에앞서서당연히삼각비의뜻부터시작하는것이첫시간입니다.오늘그첫시간함께시작해보도록할텐데요.중학교삼학년수학손샘과끝냅시다오늘공부할내용바로두가지개념인데요.오늘두가지개념첫번째제목이굉장히길어요.닮은직각삼각형에서대응변의길이의비라는것이있습니다.그다음두번째개념이바로뭐냐면삼각비라는건데.삼각비의뜻이겠죠이첫번째개념은앞으로쭉공부할내용이다직각삼각형에서주로다루어질텐데직각삼각형에서갖는기본적인성질이있는데.그성질을알고가자는삼각비를공부하기에앞선징검다리내용이다라고여러분들이받아들이시면됩니다.그렇다면닮은직각삼각형에서라고했으니까닮은직각삼각형을그려봐야되겠죠닮은직각삼각형두개를편하게그릴수있는제일좋은방법은뭘까생각해보면우리는이렇게그리면될거같아요.이렇게한번그려보겠습니다.이렇게여기에서부터수선을이렇게그어보겠습니다.일단바깥쪽에직각삼각형하나보입니다.그렇죠또다른직각삼각형를빗변위한점을잡아서이렇게수선을똑같이그어주면바로두직각삼각형에이비씨하고에이디이는서로어떤닮음에이에이닮음이된다이건우리가중학교이학년때배웠죠그렇죠각에이가공통이고그다음에둘다직각을갖고있으니까에이에이닮음이될테고우리가중이때배웠던닮음비라는건뭐였냐하면대응하는변의길이의비였어요.그렇죠그래서만약에이길이를오라고해보고이길이를십이라고해보면두삼각형의닮음비는오대십일대이겠죠그렇다면우리가이길이를사라고놔보면이길이는얼마가됩니까.당연히그렇죠사의두배인

In [79]:
ko_pre_GOOGLE

'반갑습니다.이비에스여러분들여러분들과함께중학뉴런수학삼할을강의하기됨손성민입니다.이학년이런을통해서도만났었는데.여러분들이렇게다시만나게되어서정말반갑습니다.앞으로삼과정어떻게진행되는지에대해서는여러분들께서오리엔테이션강의를통해서확인할수있을거구요.오리엔테이션강의는반드시듣고오셔야여러분들이삼학년과정이하마종이접기어떻게수업되는지확인할수있을겁니다.일단삼학년가정같은경우에는크게세개의대단원으로이루어져있는데요.첫번째가삼각비라는단어이거두번째가원의성질세번째가통계이렇게이루어져있습니다.우리가일학년하도마찬가지고이학년하도그래왔듯주로도형에대한이야기가삼화많이다루어지는구나라는걸생각하면될테구요.오늘은그첫시간이니까.삼각비삼각알아요.알아요.근데.이제합쳐져있는삼각비라는새로운영화이개념은공부하기앞서서당연히삼각비의뜻부터시작하는것이첫시간입니다.오늘도차시간함께시작해보도록할텐데요.중학교삼학년수학거끝냈어오늘공부한내용은바로두가지개념인데요.자오늘두가지그냥첫번째제목이굉장히길어요.닮은집값삼각형에서대응변의길이의피나는것이있습니다.그냥두번째개념이바로뭐냐면나는건데.생각뜻이겠죠이천번째개념은두번째앞으로쭉공부할내용이다직각삼각형에서주로다루어질텐데.직각삼각형에서가는기본적인성질이있는데.왜성질을알고가자나는어떤상각비를공부하기앞선징검다리내용이라고여러분들이받아들이셨습니다.자그렇다면닮은직각삼각형에서있으니까닮은직각삼각형그려봐야되겠죠닮은직각삼각형두개를편하게그릴수있는제일좋은방법은뭘까생각해보면우리는이렇게그러면될거같아요.차이렇게한번그려보겠습니다.이렇게이렇게여기서부터수선을이렇게커보겠습니다.일단바깥쪽에직각삼각형않아보입니다.그죠또다른직각삼각형하나를빗변이일점을잡아서이렇게수선을똑같이보여주면바로이차각성가평에이비에이디는서로어떤닮은에이닮음이된다이건우리가중학교이학년때가게가통이고그다음에둘다치과를갖고있으니까에이삶이될테고우리가주의때문에어떤닮음비일하는건뭐냐면대응하는변의길이의비왔어요.그래서그래서만약에이게들을오라고해보고얘기를십일분삼각형의닮음비는오대십일대이겠죠자그렇다면우리가이길을사라고나보면이게된얼마나됩니까.당연히그쵸오일팔일해야돼진짜이길을걷고우리가육이라고해보겠습니다.그럼거꾸로작은쪽은얼마나됐다일대이대

In [80]:
kkma = Kkma()

In [81]:
ebs_pos = kkma.pos(ko_pre_EBS)

In [ ]:
google_pos = kkma.pos(ko_pre_GOOGLE)

In [ ]:
def selected_pos(pos_text):
    alist = []
    for _ in pos_text:
        if _[1] in ['NNG','NNP','NNB','NR','NP', 'VV','VA','XR']:
            alist.append(_[0])
            continue
    return alist

# 'NNG' : 보통명사,
# 'NNP': 고유명사,
# 'NNB': 일반의존명사,
# 'NR': 수사,
# 'NP': 대명사, 
# 'VV': 동사,
# 'VA': 형용사,
# 'XR': 어근

In [ ]:
list_google_pos = selected_pos(google_pos)

In [ ]:
list_ebs_pos = selected_pos(ebs_pos)

In [50]:
list_ebs_pos

['반갑',
 '이비',
 '에스',
 '친구',
 '여러분',
 '여러분',
 '중학',
 '뉴런',
 '수학',
 '삼하',
 '강의',
 '되',
 '손',
 '석민',
 '이',
 '학년',
 '뉴런',
 '통하',
 '만나',
 '여러분',
 '이렇',
 '만나',
 '되',
 '반갑',
 '앞',
 '삼하',
 '과정',
 '어떻',
 '진행',
 '지',
 '대하',
 '여러분',
 '오리엔테이션',
 '강의',
 '통하',
 '확인',
 '수',
 '있',
 '거',
 '오리엔테이션',
 '강의',
 '듣',
 '오시',
 '여러분',
 '삼',
 '학년',
 '과정',
 '하',
 '과정',
 '어떻',
 '수업',
 '확인',
 '수',
 '있',
 '것',
 '삼',
 '학년',
 '과정',
 '같',
 '경우',
 '크',
 '삼개',
 '대단원',
 '이루어지',
 '번째',
 '삼각비',
 '단원',
 '번째',
 '원',
 '성질',
 '번째',
 '통계',
 '이루어지',
 '일',
 '학년',
 '마찬가지',
 '학년',
 '그리하',
 '도형',
 '대하',
 '이야기',
 '삼하',
 '다루',
 '라',
 '걸',
 '생각',
 '되',
 '터',
 '오늘',
 '시간',
 '삼각비',
 '삼각',
 '알',
 '비',
 '알',
 '그',
 '합',
 '쳐지',
 '삼각비',
 '새',
 '용어',
 '나오',
 '그러',
 '개념',
 '공부',
 '앞',
 '서',
 '삼각비',
 '뜻',
 '시작',
 '것',
 '시간',
 '오늘',
 '시간',
 '시작하',
 '하',
 '터',
 '중학교',
 '삼',
 '학년',
 '수학',
 '손',
 '샘',
 '끝내',
 '공부',
 '내용',
 '가지',
 '개념',
 '오늘',
 '가지',
 '개념',
 '번째',
 '제목',
 '긷',
 '닮',
 '직각',
 '삼각형',
 '대응변',
 '길이',
 '비',
 '것',
 '있',
 '그다음',
 '번째',
 '개념'

In [54]:
morphed_EBS = []
for _ in nwhites_EBS:
    morphs = Okt().pos(_)
    morphed_EBS.append(' '.join([(_[0]) for _ in morphs if _[1] in ['Noun', 'Verb', 'Adjective']]))

In [55]:
morphed_EBS

['반갑습니다',
 '이비에스 친구 여러분 여러분 중학 뉴런 수학 를 강의 된 손 민 입니다',
 '학년 뉴런 통해 서도 만났었는데요',
 '여러분 다시 만나게 되어서 정말 반갑습니다',
 '앞 과정 어떻게 진행 되는지에 대해 서는 여러분 오리엔테이션 강의 통해 확인 할수있을거고요',
 '오리엔테이션 강의 반드시 듣고오셔야 여러분 학년 정이 과정 어떻게 수업 되는지 확인 할수있을 겁니다',
 '일단 학년 과정 같은 경우 크게 개 대단원 이루어져있는데요',
 '첫 삼각비 단원 두번째 가원 성질 세번 통계 이루어져있습니다',
 '우리 학년 하도 마찬가지 학년 하도 그래 왔듯 주로 도형 대한 이야기 다루어지는구 나라는걸 생각 될테고요',
 '오늘 시간',
 '삼각비 삼각 알아요',
 '비 알아요',
 '',
 '이제 합쳐져있는 삼각비 새로운 용어 나왔죠 그럼이 개념 공부 하기에 앞서서 당연히 삼각비 뜻 시작 것 첫 시간 입니다',
 '오늘 시간 시작 해보도록 할텐데요',
 '중학교 학년 수학 손샘 끝냅시다오 늘 공부 할내용 바로 가지 개념 요',
 '오늘 가지 개념 첫 제목 굉장히길 어요',
 '닮은 직각삼각형 대응 변 길이 의비 것 있습니다',
 '그다음 두번째 개념 이바 뭐 면 삼각비 건데',
 '삼각비 뜻 겠죠 첫 개념 앞 쭉 공부 할 내용 직각삼각형 주로 다루어질텐데 직각삼각형 갖는 기본 성질 있는데',
 '성질 알고가자는 삼각비 공부 하기에 앞선 징 검다리 내용 여러분 받아들이시면 됩니다',
 '그렇다면 닮은 직각삼각형 했으니까 닮은 직각삼각형 그려 봐야되 겠죠 닮은 직각삼각형 개 편하게 그릴수있는 제일 좋은 방법 뭘 까 생각 해보면우리 는 그리면 될거 같아요',
 '한번 그려 보겠습니다',
 '여기 수선 어보 겠습니다',
 '일단 바깥쪽 직각삼각형 보입니다',
 '그렇죠 직각삼각형 를 빗변 위 점 잡아서 수선 똑같 이그 주면 바로 직각삼각형 비씨 에이 디 서로 닮음 에이 닮음 된다 이건우 리가 중학교 학년 때 배웠죠 그렇죠 에이 공통 그다음 둘다 직각 갖

In [ ]:
# morphs 형태 :
# [('오리엔테이션', 'Noun'),
#  ('강의', 'Noun'),
#  ('는', 'Josa'),
#  ('반드시', 'Noun'),
#  ('듣고오셔야', 'Verb'),
#  ('여러분', 'Noun'),
#  ('들', 'Suffix'),
#  ('이삼', 'Noun'),
#  ('학년', 'Noun'),
#  ('과정', 'Noun'),
#  ('이하', 'Noun'),
#  ('과정', 'Noun'),
#  ('이', 'Josa'),
#  ('특히', 'Adverb'),
#  ('어떻게', 'Adjective'),
#  ('수업', 'Noun'),
#  ('되는지', 'Verb'),
#  ('확인', 'Noun'),
#  ('할수있을', 'Verb'),
#  ('겁니다', 'Verb')]

In [ ]:
EBSlist_tokens = [' '.join(Text(ma(_)).tokens) for _ in new_textE1]

In [ ]:
EBSlist_tokens

In [56]:
t = Tokenizer()
t.fit_on_texts(morphed_EBS)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 8242


In [58]:
sequences = list()

for line in morphed_EBS: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[1873, 545],
 [1873, 545, 6],
 [1873, 545, 6, 6],
 [1873, 545, 6, 6, 1874],
 [1873, 545, 6, 6, 1874, 464],
 [1873, 545, 6, 6, 1874, 464, 142],
 [1873, 545, 6, 6, 1874, 464, 142, 217],
 [1873, 545, 6, 6, 1874, 464, 142, 217, 244],
 [1873, 545, 6, 6, 1874, 464, 142, 217, 244, 465],
 [1873, 545, 6, 6, 1874, 464, 142, 217, 244, 465, 1567],
 [1873, 545, 6, 6, 1874, 464, 142, 217, 244, 465, 1567, 2397]]

In [59]:
print(t.word_index)

{'도': 1, '길이': 2, '에이': 3, '에이치': 4, '루트': 5, '여러분': 6, '문제': 7, '뭐': 8, '사인': 9, '우리': 10, '구': 11, '코사인': 12, '삼각형': 13, '높이': 14, '엑스': 15, '곱하기': 16, '탄젠트': 17, '그렇죠': 18, '빗변': 19, '얘기': 20, '밑변': 21, '여기': 22, '각': 23, '이': 24, '비': 25, '변': 26, '선생님': 27, '번': 28, '거': 29, '곱': 30, '넓이': 31, '그럼': 32, '어떻게': 33, '직각삼각형': 34, '때': 35, '한번': 36, '삼각비': 37, '지금': 38, '비씨': 39, '입니다': 40, '이각': 41, '오늘': 42, '기준': 43, '얼마': 44, '구하라': 45, '생각': 46, '값': 47, '바로': 48, '정리': 49, '이분의일': 50, '게': 51, '씨': 52, '돼': 53, '마이너스': 54, '더하기': 55, '이제': 56, '의제': 57, '이비': 58, '는': 59, '다음': 60, '좀': 61, '그림': 62, '하겠습니다': 63, '누구': 64, '겁니다': 65, '걸': 66, '것': 67, '보세요': 68, '구해': 69, '해주면': 70, '이야기': 71, '디': 72, '그다음': 73, '겠죠': 74, '표현': 75, '때문': 76, '해서': 77, '피타고라스': 78, '보면': 79, '일단': 80, '답': 81, '돼요': 82, '공식': 83, '된다': 84, '왜': 85, '제곱': 86, '보도록': 87, '첫': 88, '됐나요': 89, '당연히': 90, '해보면': 91, '조금': 92, '끼': 93, '시간': 94, '볼게요': 95, '개념': 96, '씨디': 97, '마주': 98, '됐죠': 99, '둘': 1

In [60]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : 둘다


In [61]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 100


In [62]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
  1873  545]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0

In [63]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [64]:
print(X[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
  1873]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0

In [65]:
print(y[:3]) # 레이블

[545   6   6]


In [66]:
vocab_size

8242

In [67]:
y = to_categorical(y, num_classes=vocab_size)

In [68]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=2)

Epoch 1/10
2224/2224 - 230s - loss: 7.0065 - accuracy: 0.0231
Epoch 2/10
2224/2224 - 230s - loss: 6.4792 - accuracy: 0.0414
Epoch 3/10
2224/2224 - 234s - loss: 6.1069 - accuracy: 0.0768
Epoch 4/10
2224/2224 - 235s - loss: 5.7930 - accuracy: 0.1025
Epoch 5/10
2224/2224 - 243s - loss: 5.5240 - accuracy: 0.1204
Epoch 6/10
2224/2224 - 241s - loss: 5.2806 - accuracy: 0.1352
Epoch 7/10
2224/2224 - 239s - loss: 5.0569 - accuracy: 0.1490
Epoch 8/10
2224/2224 - 240s - loss: 4.8513 - accuracy: 0.1612
Epoch 9/10
2224/2224 - 241s - loss: 4.6590 - accuracy: 0.1748
Epoch 10/10
2224/2224 - 240s - loss: 4.4789 - accuracy: 0.1899


In [69]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [71]:
sentence_generation(model, t, '이 병 분에 높이', 5)

'이 병 분에 높이 에이치 에이치 에이치 에이치 에이치'